In [ ]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [ ]:
import torchvision 
import torch.nn as nn
import torch.optim as optim 
from torchvision import datasets , models , transforms 
from torch.optim import lr_scheduler 
import numpy as np 
import matplotlib.pyplot as plt 
import time 
import os 
import copy 

In [ ]:
# As VGG16 requires the data in certain form so ,we are transforming the data as per required(normalization)
train_datatransform = transforms.Compose([transforms.RandomResizedCrop(224),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485 , 0.456 ,0.406] ,[0.229 , 0.224 , 0.225])])
test_datatrannsform = transforms.Compose([transforms.RandomResizedCrop(224), 
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(), 
                                         transforms.Normalize([0.485 ,0.456 ,0.406] ,[0.229 , 0.224 ,0.225])])

In [ ]:
#Yeah , we need to export the data that we have created 
train_image_dataset = datasets.ImageFolder(root='./Visiondataset/train',transform=train_datatransform )
test_image_dataset = datasets.ImageFolder(root ='./Visiondataset/test', transform=test_datatrannsform)

In [ ]:
#DataLoader - Load the train and test data in the form of batches

train_dataloader = torch.utils.data.DataLoader(train_image_dataset , batch_size =4 ,shuffle=True , num_workers=4 )
test_dataloader = torch.utils.data.DataLoader(test_image_dataset , batch_size = 4 , shuffle= True , num_workers=4)

In [ ]:
class_names = train_image_dataset.classes
print(class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
import PIL

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_dataloader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])


In [ ]:
def train_model(model , criterion , optimizer , scheduler ,num_epochs):
  since =time.time()
  best_acc =0.0
  best_model_wts = copy.deepcopy(model.state_dict())
  
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch , num_epochs -1))
    print('-'*10)
    
    scheduler.step()
    model.train()
    
    running_loss = 0.0 
    running_corrects = 0
    
    for inputs , labels in train_dataloader:
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      optimizer.zero_grad()
      
      outputs = model(inputs)
      _, preds = torch.max(outputs , 1)
      loss = criterion(outputs , labels)
      
      loss.backward()
      optimizer.step()
      
      running_loss += loss.item() *inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
      
    epoch_loss = running_loss / len(train_image_dataset)
    epoch_acc = running_corrects.double() / len(train_image_dataset)
    
    print('Loss : {}  Acc :{}'.format(epoch_loss , epoch_acc))
    
    if epoch_acc > best_acc:
      best_acc =epoch_acc 
      best_model_wts = copy.deepcopy(model.state_dict())
    
  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    
    
  model.load_state_dict(best_model_wts)
  return model

In [ ]:
def set_require_grad(model , feature_extract):
  if feature_extract:
    for param in model.parameters():
      param.require_grad=False